# 📘 Notion Markdown QA 임베딩 파이프라인
이 노트북은 Notion에서 Export한 Markdown 파일들을 병합하고, 각 문단으로부터 질문-답변(QA) 쌍을 생성하여 Chroma 벡터 DB에 저장합니다.

In [4]:
import os
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

## 1️⃣ Markdown 병합 함수

In [5]:
def merge_markdown_files(base_dir: str) -> str:
    merged = ""
    for root, _, files in os.walk(base_dir):
        for file in files:
            if file.endswith(".md"):
                path = os.path.join(root, file)
                with open(path, "r", encoding="utf-8") as f:
                    content = f.read()
                    merged += f"\n\n# {file}\n{content}"
    return merged

## 2️⃣ 텍스트 분할 함수

In [33]:
def split_by_title(text: str):
    chunks = text.split("\n# ")
    cleaned = ["# " + c.strip() if not c.startswith("#") else c.strip() for c in chunks if c.strip()]
    return cleaned


## 3️⃣ 질문 생성 체인 함수 (답변 → 예상 질문)

In [ ]:
def build_qa_chain():
    prompt = PromptTemplate(
        input_variables=["answer"],
        template="""
아래는 고객에게 제공된 안내문입니다. 고객이 이 내용을 받기 전에 어떤 질문을 했을지 유추하여 "Q:"로 시작하는 질문과 "A:"로 시작하는 답변을 구성하세요.

답변:
{answer}

형식:
Q: (예상 질문)
A: (원본 답변)
"""
    )
    llm = ChatOpenAI(model_name="gpt-4", temperature=0.3)
    return LLMChain(prompt=prompt, llm=llm)


In [ ]:

def build_qa_chain_json():
    prompt = PromptTemplate(
        input_variables=["answer"],
        template="""
아래는 고객에게 제공된 안내문입니다. 고객이 이 내용을 받기 전에 어떤 질문을 했을지 유추하여 JSON 형식으로 반환하세요.

반환 형식:
{{
  "faq_id": "FAQ###",       // 고유 ID는 FAQ001부터 자동 증가하거나 생략 가능
  "category": "",           // 적절한 카테고리를 추정해서 넣어주세요 (예: 배송, 교환, 설치 등)
  "question": "",           // 유추된 질문
  "answer": "",             // 원본 답변
  "keywords": ""            // 질문/답변에 포함된 핵심 키워드 (쉼표로 구분)
}}

답변:
{answer}
"""
    )
    llm = ChatOpenAI(model_name="gpt-4", temperature=0.3)
    return LLMChain(prompt=prompt, llm=llm)


## 4️⃣ QA Document 생성 함수

In [8]:
def generate_documents(chunks, chain):
    documents = []
    for chunk in chunks:
        qa = chain.run(answer=chunk)
        documents.append(Document(page_content=qa, metadata={}))
    return documents

## 5️⃣ Chroma에 저장 함수

In [9]:
def store_to_chroma(documents, persist_dir="./chroma_notion_qa"):
    embeddings = OpenAIEmbeddings()
    db = Chroma.from_documents(documents, embeddings, persist_directory=persist_dir)
    db.persist()
    return db

## ✅ 실행: 병합 → 분할 → QA 생성 → 저장

In [27]:
from pathlib import Path
import os

base_dir = Path.cwd().parents[1] / "SKN13-3rd-4TEAM" / "data" / "raw_docs" / "qna_raw"
print(f"🔍 탐색 경로: {base_dir}")

merged_text = ""

for root, _, files in os.walk(base_dir):
    for file in files:
        if file.lower().endswith(".md"):
            path = os.path.join(root, file)
            print(f"📄 병합 중: {path}")
            with open(path, "r", encoding="utf-8") as f:
                content = f.read()
                merged_text += f"\n\n{content}"

print("✅ 병합 완료" if merged_text.strip() else "⚠ 병합된 내용 없음")


🔍 탐색 경로: c:\Users\Playdata\Documents\Project\SKN13-3rd-4TEAM\data\raw_docs\qna_raw
📄 병합 중: c:\Users\Playdata\Documents\Project\SKN13-3rd-4TEAM\data\raw_docs\qna_raw\(구매 후 바로) 양품, 검수해서 보내주세요 21f032cbb6d481a0bfd8e10d91cbe58e.md
📄 병합 중: c:\Users\Playdata\Documents\Project\SKN13-3rd-4TEAM\data\raw_docs\qna_raw\7179 내부 사이즈 21f032cbb6d48159b4e7e747bf595f29.md
📄 병합 중: c:\Users\Playdata\Documents\Project\SKN13-3rd-4TEAM\data\raw_docs\qna_raw\LT2 업데이트 파일 21f032cbb6d481ffb2d2c0d29b71cd34.md
📄 병합 중: c:\Users\Playdata\Documents\Project\SKN13-3rd-4TEAM\data\raw_docs\qna_raw\LT2 업데이트 해도 안됩니다 21f032cbb6d4810b9b61e36581e8167c.md
📄 병합 중: c:\Users\Playdata\Documents\Project\SKN13-3rd-4TEAM\data\raw_docs\qna_raw\X6 어플 업데이트 21f032cbb6d481ee9b0ad444f1151f54.md
📄 병합 중: c:\Users\Playdata\Documents\Project\SKN13-3rd-4TEAM\data\raw_docs\qna_raw\X6 어플 업데이트 해도 안됩니다 21f032cbb6d48186b513e1d4e3cd90c8.md
📄 병합 중: c:\Users\Playdata\Documents\Project\

In [25]:
root_dir = Path.cwd().parents[0]
root_dir

WindowsPath('c:/Users/Playdata/Documents/Project/SKN13-3rd-4Team')

In [28]:
# 병합된 내용을 저장할 경로 설정
output_path = root_dir / "data" / "merged_docs" / "merged.md"

# 폴더 없으면 생성
output_path.parent.mkdir(parents=True, exist_ok=True)

# 파일로 저장
with open(output_path, "w", encoding="utf-8") as f:
    f.write(merged_text)

print(f"✅ 병합 완료: 저장 위치 → {output_path}")

✅ 병합 완료: 저장 위치 → c:\Users\Playdata\Documents\Project\SKN13-3rd-4Team\data\merged_docs\merged.md


In [21]:
len(merged_text)

87572

In [34]:

print("✂️ 텍스트 분할 중...")
chunks = split_by_title(merged_text)


✂️ 텍스트 분할 중...


In [37]:
chunks[:20]

['# (구매 후 바로) 양품, 검수해서 보내주세요.\n\n생성일: 2025년 6월 27일 오전 10:09\n유형: 배송\n해외: 국내, 해외\n\n안녕하세요\n구매해주셔서 감사합니다.\n양품으로 최대한 빠르게 보내드리도록 노력하겠습니다.\n감사합니다:)',
 '# 7179 내부 사이즈\n\n생성일: 2025년 6월 27일 오전 10:09\n유형: 7179, 상품\n해외: 국내\n\n안녕하세요.\n소형 내부 높이는 22cm, 가로 27.8cm, 세로 20.5cm 입니다.\n대형 내부 높이는 26.5cm, 가로 30.4cm, 세로 20.5cm 입니다.\n감사합니다.',
 '# LT2 업데이트 파일\n\n생성일: 2025년 6월 27일 오전 10:09\n유형: 빔프로젝트\n해외: 해외\n\n안녕하세요.\nLT2 경우 앱스토어에서 어플 업데이트 해주시면 기본적으로 다시 ott 버전이 업데이트가 됩니다.\n만약 앱스토어에서 재 설치를 하였는데도 ott가 접속이 안되신다면 아래의 최신 업데이트 파일 다운로드 해서 재설치 해보시기 바랍니다.\n[https://drive.google.com/file/d/1eZBaBn9W6Nt48jDkSZ3sCmUJJ3IOBEDp/view](https://drive.google.com/file/d/1eZBaBn9W6Nt48jDkSZ3sCmUJJ3IOBEDp/view)\n구글 드라이브 링크입니다.\nzip안에 설치방법 확인하신 후 설치 부탁드립니다.\n감사합니다.',
 '# LT2 업데이트 해도 안됩니다.\n\n생성일: 2025년 6월 27일 오전 10:09\n유형: 빔프로젝트\n해외: 해외\n\n안녕하세요.\nLT2 경우 앱스토어에서 어플 업데이트 해주시면 기본적으로 다시 ott 버전이 업데이트가 됩니다.\n만약 앱스토어에서 재 설치를 하였는데도 ott가 접속이 안되신다면 아래의 최신 업데이트 파일 다운로드 해서 재설치 해보시기 바랍니다.\n[https://drive.google.com/file/d/1eZBaBn9W6Nt48jDkSZ3sCm

In [38]:

print("💡 질문 생성 중...")
chain = build_qa_chain()
docs = generate_documents(chunks, chain)


💡 질문 생성 중...


C:\Users\Playdata\AppData\Local\Temp\ipykernel_18832\2990547196.py:15: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-4", temperature=0.3)
C:\Users\Playdata\AppData\Local\Temp\ipykernel_18832\2990547196.py:16: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  return LLMChain(prompt=prompt, llm=llm)
C:\Users\Playdata\AppData\Local\Temp\ipykernel_18832\804560214.py:4: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  qa = chain.run(answer=chu

In [40]:
for i, doc in enumerate(docs[:15]):  # 앞부분만 샘플 확인
    print(f"\n🔹 Document {i+1}:\n{doc.page_content}\n{'-'*60}")


🔹 Document 1:
Q: 제가 구매한 제품을 검수 후에 보내주실 수 있나요?
A: 안녕하세요
구매해주셔서 감사합니다.
양품으로 최대한 빠르게 보내드리도록 노력하겠습니다.
감사합니다:)
------------------------------------------------------------

🔹 Document 2:
Q: 7179 상품의 소형과 대형 내부 사이즈가 어떻게 되나요?
A: 안녕하세요.
소형 내부 높이는 22cm, 가로 27.8cm, 세로 20.5cm 입니다.
대형 내부 높이는 26.5cm, 가로 30.4cm, 세로 20.5cm 입니다.
감사합니다.
------------------------------------------------------------

🔹 Document 3:
Q: LT2 앱의 ott 버전이 업데이트가 안되는데 어떻게 해야 하나요?
A: LT2 경우 앱스토어에서 어플 업데이트 해주시면 기본적으로 다시 ott 버전이 업데이트가 됩니다. 만약 앱스토어에서 재 설치를 하였는데도 ott가 접속이 안되신다면 아래의 최신 업데이트 파일 다운로드 해서 재설치 해보시기 바랍니다. [https://drive.google.com/file/d/1eZBaBn9W6Nt48jDkSZ3sCmUJJ3IOBEDp/view](https://drive.google.com/file/d/1eZBaBn9W6Nt48jDkSZ3sCmUJJ3IOBEDp/view) 구글 드라이브 링크입니다. zip안에 설치방법 확인하신 후 설치 부탁드립니다. 감사합니다.
------------------------------------------------------------

🔹 Document 4:
Q: LT2를 업데이트하려고 하는데 작동이 안되네요. 어떻게 해야하나요?
A: 안녕하세요. LT2 경우 앱스토어에서 어플 업데이트 해주시면 기본적으로 다시 ott 버전이 업데이트가 됩니다. 만약 앱스토어에서 재 설치를 하였는데도 ott가 접속이 안되신다면 아래의 최신 업데이트 파

In [42]:

print("📦 Chroma 저장 중...")
store_to_chroma(docs, persist_dir="../data/vectordb_chroma")

print("✅ 완료: 총 문서 수 =", len(docs))

📦 Chroma 저장 중...
✅ 완료: 총 문서 수 = 200


In [6]:
#OCR테스트 
%pip install google-cloud-vision


  Using cached proto_plus-1.26.1-py3-none-any.whl.metadata (2.2 kB)
  Using cached protobuf-6.31.1-cp310-abi3-win_amd64.whl.metadata (593 bytes)
   ---------------------------------------- 0.0/4.3 MB ? eta -:--:--
   ---------------------------------------- 4.3/4.3 MB 64.6 MB/s eta 0:00:00
Using cached proto_plus-1.26.1-py3-none-any.whl (50 kB)
Using cached protobuf-6.31.1-cp310-abi3-win_amd64.whl (435 kB)

  Attempting uninstall: protobuf

    Found existing installation: protobuf 5.29.3

    Uninstalling protobuf-5.29.3:

      Successfully uninstalled protobuf-5.29.3

   ---------------------------------------- 0/4 [protobuf]
   ---------------------------------------- 0/4 [protobuf]
   ---------------------------------------- 0/4 [protobuf]
   ---------------------------------------- 0/4 [protobuf]
   ---------------------------------------- 0/4 [protobuf]
   ---------------------------------------- 0/4 [protobuf]
   ---------------------------------------- 0/4 [protobuf]
   ------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unstructured-inference 1.0.5 requires accelerate, which is not installed.
unstructured-inference 1.0.5 requires pdfminer-six, which is not installed.
unstructured-inference 1.0.5 requires pypdfium2, which is not installed.
unstructured-inference 1.0.5 requires rapidfuzz, which is not installed.
opentelemetry-proto 1.34.1 requires protobuf<6.0,>=5.0, but you have protobuf 6.31.1 which is incompatible.


In [10]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "credential.json"

In [11]:
from google.cloud import vision
from google.cloud.vision_v1 import types

def google_vision_ocr(image_path):
    client = vision.ImageAnnotatorClient()
    
    with open(image_path, 'rb') as img_file:
        content = img_file.read()

    image = vision.Image(content=content)
    response = client.text_detection(image=image)
    texts = response.text_annotations

    if not texts:
        return "❌ 텍스트 인식 실패"

    return texts[0].description.strip()



In [ ]:
result = google_vision_ocr("test.jpg")
print("🧾 OCR 결과:\n", result)


FileNotFoundError: [Errno 2] No such file or directory: 'text.jpg'